<a href="https://colab.research.google.com/github/sanskarpyml/kaggle-LLM-Classification-Finetuning/blob/main/LLM_CLASSIFICATION_FINETUNING_CLEAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#importing dependencies


In [ ]:
import os
import json
from zipfile import ZipFile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Data curation


In [ ]:
kaggle_credentials =  json.load(open('kaggle.json'))

In [ ]:
os.environ['KAGGLE_USERNAME'] = kaggle_credentials['username']
os.environ['KAGGLE_KEY'] = kaggle_credentials['key']

In [ ]:
!kaggle competitions download -c llm-classification-finetuning

llm-classification-finetuning.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
with ZipFile('llm-classification-finetuning.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
dataset = pd.read_csv("train.csv")

In [ ]:
# Step 1: Create single label column
def get_label(row):
    if row["winner_model_a"] == 1:
        return 0
    elif row["winner_model_b"] == 1:
        return 1
    else:
        return 2

In [ ]:
dataset["label"] = dataset.apply(get_label, axis=1)

In [ ]:
dataset.head()

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,label
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0,1
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1,2
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0,1


In [ ]:
# Step 2: Concatenate input into a single text field
dataset['text'] = dataset.apply(
    lambda row: f"Prompt: {row['prompt']} [SEP] Response A: {row['response_a']} [SEP] Response B: {row['response_b']}",
    axis=1
)

In [ ]:
!pip install transformers datasets

In [ ]:
from transformers import RobertaTokenizer

# Load the tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Example: Tokenize a single sentence (just to see how it works)
# encoded = tokenizer("Hello world", truncation=True, padding="max_length", max_length=512)
# print(encoded)

In [ ]:
from datasets import Dataset

# Assuming your DataFrame is called df and has 'text' and 'label' columns
hf_dataset = Dataset.from_pandas(dataset[['text', 'label']])

In [ ]:
def tokenize_function(example):
    return tokenizer(
        example["text"],
        truncation=True,           # Cut off texts longer than max_length
        padding="max_length",      # Pad all to the same length
        max_length=216             # You can reduce to 256 for faster training
    )

In [ ]:
tokenized_dataset = hf_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/57477 [00:00<?, ? examples/s]

In [ ]:
from datasets import DatasetDict

# Split into 90% training and 10% validation
split_dataset = tokenized_dataset.train_test_split(test_size=0.1, seed=42)

# Optional: rename for clarity
split_dataset = DatasetDict({
    'train': split_dataset['train'],
    'validation': split_dataset['test']
})


In [ ]:
from transformers import AutoModelForSequenceClassification

# Load RoBERTa-base with 3 output labels
model = AutoModelForSequenceClassification.from_pretrained(
    "roberta-base",
    num_labels=3
)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir='./logs',
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_total_limit=1,
    fp16 = True
)

In [ ]:
!pip install evaluate

In [ ]:
import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, label = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=label)


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=split_dataset["train"],
    eval_dataset=split_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/tmp/ipython-input-29-3754254662.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sanskariitb (sanskariitb-indira-gandhi-national-open-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,1.083400,1.079969,0.414927
2,1.052400,1.065595,0.425017


TrainOutput(global_step=6468, training_loss=1.0732002659351205, metrics={'train_runtime': 1422.3806, 'train_samples_per_second': 72.736, 'train_steps_per_second': 4.547, 'total_flos': 1.1483938675393632e+16, 'train_loss': 1.0732002659351205, 'epoch': 2.0})

In [ ]:
trainer.evaluate()


{'eval_loss': 1.0655945539474487,
 'eval_accuracy': 0.42501739735560196,
 'eval_runtime': 18.6688,
 'eval_samples_per_second': 307.893,
 'eval_steps_per_second': 19.283,
 'epoch': 2.0}

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import Trainer
import pandas as pd
from datasets import Dataset

# Load test data
test_df = pd.read_csv("test.csv")

# Preprocess test data
test_df['text'] = test_df.apply(
    lambda row: f"Prompt: {row['prompt']} [SEP] Response A: {row['response_a']} [SEP] Response B: {row['response_b']}",
    axis=1
)

# Convert to HuggingFace Dataset
hf_test_dataset = Dataset.from_pandas(test_df[['id', 'text']])

# Tokenize
tokenized_test_dataset = hf_test_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Create DataLoader
test_dataloader = DataLoader(tokenized_test_dataset, batch_size=16, shuffle=False)

# Move model to GPU (optional)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Inference
all_logits = []
for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
        all_logits.append(outputs.logits)

# Concatenate results
logits = torch.cat(all_logits, dim=0)
probabilities = torch.softmax(logits, dim=-1).cpu().numpy()

# Create submission file
submission_df = pd.DataFrame(probabilities, columns=["winner_model_a", "winner_model_b", "winner_tie"])
submission_df.insert(0, "id", test_df["id"])
submission_df.to_csv("submission.csv", index=False)


Map:   0%|          | 0/3 [00:00<?, ? examples/s]